In [1]:
import sys
sys.path.append(r"../")
#sys.path.append(r"../utils/data_manipulation")
import pandas as pd
from utils.data_manipulation.data_imputation import impute_from_column

In [2]:
df = pd.read_csv(r"..\DeppClinic_prediction_task.csv")

columns = ["id", "age_child_pre"]

## Intake time

In [4]:
df["intake_date"] =  pd.to_datetime(df["sciafca_timestamp_intake_arm_1"], errors='coerce')
columns.append("intake_date")

## ER time

In [5]:
df = impute_from_column(df, "complaint_date", "chameleon_suicide_er_date_stu_intake_arm_1")
df["ER_date_time1"] = df["complaint_date"]
columns.append("ER_date_time1")

###  ER date

In [6]:
time2_er_dates = [f"chameleon_suicide_er_date_stu_{event}" for event in redcap_event_names]
for er_time in time2_er_dates:
    df[er_time] = pd.to_datetime(df[er_time], errors='coerce')
df["ER_date_target"] = df[time2_er_dates].min(axis=1)
columns.append("ER_date_target")

## Psych ward

In [7]:
time2_psych_dates = [f"chameleon_psychiatric_date_stu_{event}" for event in redcap_event_names]
for psych_time in time2_psych_dates:
    df[psych_time] = pd.to_datetime(df[psych_time], errors='coerce')
    
df["psychiatric_date_target"] = df[time2_psych_dates].min(axis=1)
columns.append("psychiatric_date_target")

## Attempt


In [8]:
time2_visit_date_dates = [f"visit_date_stu_{event}" for event in redcap_event_names]
time2_is_attempt = [f"chameleon_attempt_stu_{event}" for event in redcap_event_names]

In [9]:
for visit_date, is_attempt in zip(time2_visit_date_dates, time2_is_attempt):
    df.loc[df[is_attempt] != 1, visit_date] = pd.NaT
    df[visit_date] = pd.to_datetime(df[visit_date], errors='coerce')

In [10]:
df["suicide_attempt_date_target"] = df[time2_visit_date_dates].min(axis=1)
columns.append("suicide_attempt_date_target")

## Save

In [11]:
sci_intake = [f"{q}_intake_arm_1" for q in sci_af_ca]
columns.extend(sci_intake)

In [12]:
df[columns].to_csv("data_for_survival_analysis.csv")